In [4]:
import signal
import pyttsx3
from pylsl import StreamInlet, resolve_stream
import numpy as np
import time 
from time import sleep
import matplotlib.pyplot as plt
from matplotlib import style 
from collections import deque
from numpy.fft import fft, fftfreq
from scipy import signal
import scipy.fft
from scipy.signal import butter, lfilter
from keras.models import load_model
import pandas as pd
from scipy.stats import skew
from scipy.signal import welch
from scipy import stats
import serial

arduino1 = serial.Serial('COM8', 9600 , timeout = 1)
arduino2 = serial.Serial('COM11', 9600 , timeout = 1)
time.sleep(2)

modelA_R = load_model('../CODIGOS_FINALES/Red_neuronal_arribaabajoruben1.h5')
modelD_I = load_model('../CODIGOS_FINALES/Red_neuronal_derechaizquierdaruben.h5')

book1 = open(r"../TIG FINAL/Sonidos/Binevenida.txt")
book2 = open(r"../TIG FINAL/Sonidos/derecha.txt")
book3 = open(r"../TIG FINAL/Sonidos/abajo.txt")
book4 = open(r"../TIG FINAL/Sonidos/izquierda.txt")
book5 = open(r"../TIG FINAL/Sonidos/arriba.txt")
book6 = open(r"../TIG FINAL/Sonidos/fin.txt")
book1_text = book1.readlines()
book2_text = book2.readlines()
book3_text = book3.readlines()
book4_text = book4.readlines()
book5_text = book5.readlines()
book6_text = book6.readlines()

engine = pyttsx3.init()
for line in book1_text:
    engine.say(line)
    engine.runAndWait()

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandpass')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def redArriba_abajo(test):
    test.iloc[:,:] = test
    test = test.to_numpy()
    test = np.asarray(test).astype(np.float32)
    resultado = modelA_R.predict(test).round()

    return resultado

def redderecha_izquierda(test):
    test.iloc[:,:] = test
    test = test.to_numpy()
    test = np.asarray(test).astype(np.float32)
    resultado = modelD_I.predict(test).round()
    
    return resultado

def calculo_caracte2(sen):
    
    fs = 250
    lowcut = 2
    highcut = 30

    sig=butter_bandpass_filter(sen , lowcut, highcut, fs,order=6)

    #medidas estadisticas 
    mean_values = []
    std_values = []
    energy_values = []
    curtosis = []
    skewness = []

    #medidas temporales 
    RMS_Value = []
    

    #medidas espectrales
    
    ancho_banda = []

    # Media
    mean = np.mean(sig)
    mean_values.append(mean)

    # Desviación estándar
    std = np.std(sig)
    std_values.append(std)

    # Energía
    energy = np.sum(np.square(sig)) / len(sig)
    energy_values.append(energy)

    #curtosis
    curtosis = stats.kurtosis(sig)

    #skewness
    skewness = skew(sig)

    #valor de rms
    RMS_Value = np.sqrt(np.mean(sig**2))

    #cruces por 0
    cruces_por_cero = np.where(np.diff(np.signbit(sig)))[0]
    contador_cruces = len(cruces_por_cero)

    #periodograma de welch
    frecuencias, periodograma = welch(sig)
    media_periodograma = np.mean(periodograma)

    #Ancho de banda 
    n = len(sig)
    frecuencias = fftfreq(n)
    transformada = fft(sig)
    transformada_abs = np.abs(transformada) # Encontrar el índice de la frecuencia máxima
    indice_max = np.argmax(transformada_abs)# Obtener la frecuencia correspondiente al índice máximo
    frecuencia_max = frecuencias[indice_max]# Calcular el ancho de banda
    ancho_banda = 2 * np.abs(frecuencia_max)

    caracteristicas = [mean,std,energy,curtosis,skewness,RMS_Value,contador_cruces,media_periodograma,ancho_banda]

    carac = pd.DataFrame(columns=['mean', 'std', 'energy', 'curtosis','skewness','RMS','#cruce_0','media_welch','ancho_banda'])
    carac.loc[1, 'mean'] = caracteristicas[0]
    carac.loc[1, 'std'] = caracteristicas[1]
    carac.loc[1, 'energy'] = caracteristicas[2]
    carac.loc[1, 'curtosis'] = caracteristicas[3]
    carac.loc[1, 'skewness'] = caracteristicas[4] 
    carac.loc[1, 'RMS'] = caracteristicas[5]
    carac.loc[1, '#cruce_0'] = caracteristicas[6]
    carac.loc[1, 'media_welch'] = caracteristicas[7]
    carac.loc[1, 'ancho_banda'] = caracteristicas[8]

    
    return carac 



while True:
    
    sleep(5)

    f_max = 60
    segundos = 100
    interacciones = segundos * 25

    last_print = time.time()
    fps_counter = deque(maxlen=150)

    # LECTURA DE DERECHA 
    for line in book2_text:
        engine.say(line)
        engine.runAndWait()

    sleep(8)

    print("buscando la transmicion EEG de la derecha")
    streams = resolve_stream('type','EEG')

    inlet = StreamInlet(streams[0])

    channel_data1 = {}

    for i in range(interacciones):
        for i in range(3):
            sample, timestamp = inlet.pull_sample()
            if i not in channel_data1:
                channel_data1[i] = sample 
            else:
                channel_data1[i].append(sample[:f_max])

        fps_counter.append(time.time() - last_print)
        last_print = time.time()
        cur_raw_hz = 1/(sum(fps_counter)/len(fps_counter))

    channel_011 = channel_data1[0]
    channel_012 = channel_data1[1]
    channel_013 = channel_data1[2]

    cannal_011 = []
    cannal_012 = []
    cannal_013 = []

    c_0 = channel_011[8:]
    c_0 = np.asarray(c_0)
    for i in range(495):
        for j in range(4):
            cannal_011.append(c_0[i][j])

    c_0 = channel_012[8:]
    c_0 = np.asarray(c_0)
    for i in range(495):
        for j in range(4):
            cannal_012.append(c_0[i][j])

    c_0 = channel_013[8:]
    c_0 = np.asarray(c_0)
    for i in range(495):
        for j in range(4):
            cannal_013.append(c_0[i][j])

    caracte = calculo_caracte2(cannal_011)
    x = redderecha_izquierda(caracte)

    if x == 0:

        distancia = arduino2.readline()

        print("DERECHA")
        arduino1.write(b'6')

        if distancia == "Continue":
            arduino1.write(b'6')

        elif distancia == "Stop":
            arduino1.write(b'1')

        sleep(4)
        arduino1.write(b'1')


    elif x == 1:

        distancia = arduino2.readline()

        print("IZQUIERDA")
        arduino1.write(b'5')

        if distancia == "Continue":
            arduino1.write(b'5')

        elif distancia == "Stop":
            arduino1.write(b'1')

        sleep(4)
        arduino1.write(b'1')


    sleep(2)

    # LECTURA DE ABAJO 

    for line in book3_text:
        engine.say(line)
        engine.runAndWait()

    sleep(8)

    print("buscando la transmicion EEG de abajo")
    streams = resolve_stream('type','EEG')

    inlet = StreamInlet(streams[0])

    channel_data2 = {}

    for i in range(interacciones):
        for i in range(3):
            sample, timestamp = inlet.pull_sample()
            if i not in channel_data2:
                channel_data2[i] = sample 
            else:
                channel_data2[i].append(sample[:f_max])

        fps_counter.append(time.time() - last_print)
        last_print = time.time()
        cur_raw_hz = 1/(sum(fps_counter)/len(fps_counter))

    channel_021 = channel_data2[0]
    channel_022 = channel_data2[1]
    channel_023 = channel_data2[2]

    cannal_021 = []
    cannal_022 = []
    cannal_023 = []

    c_0 = channel_021[8:]
    c_0 = np.asarray(c_0)
    for i in range(495):
        for j in range(4):
            cannal_021.append(c_0[i][j])

    c_0 = channel_022[8:]
    c_0 = np.asarray(c_0)
    for i in range(495):
        for j in range(4):
            cannal_022.append(c_0[i][j])

    c_0 = channel_023[8:]
    c_0 = np.asarray(c_0)
    for i in range(495):
        for j in range(4):
            cannal_023.append(c_0[i][j])

    caracte = calculo_caracte2(cannal_021)
    x = redArriba_abajo(caracte)

    if x == 1:

        distancia = arduino2.readline()

        print("ARRIBA")
        arduino1.write(b'2')

        if distancia == "Continue":
            arduino1.write(b'2')

        elif distancia == "Stop":
            arduino1.write(b'1')

        sleep(4)
        arduino1.write(b'1')

    elif x == 0:

        distancia = arduino2.readline()

        print("ABAJO")
        arduino1.write(b'3')

        if distancia == "Continue":
            arduino1.write(b'3')

        elif distancia == "Stop":
            arduino1.write(b'1')

        sleep(4)
        arduino1.write(b'1')

    sleep(2)
    # LECTURA DE IZQUIERDA

    for line in book4_text:
        engine.say(line)
        engine.runAndWait()

    sleep(8)

    print("buscando la transmicion EEG de la izquierda")
    streams = resolve_stream('type','EEG')

    inlet = StreamInlet(streams[0])

    channel_data3 = {}

    for i in range(interacciones):
        for i in range(3):
            sample, timestamp = inlet.pull_sample()
            if i not in channel_data3:
                channel_data3[i] = sample 
            else:
                channel_data3[i].append(sample[:f_max])

        fps_counter.append(time.time() - last_print)
        last_print = time.time()
        cur_raw_hz = 1/(sum(fps_counter)/len(fps_counter))

    channel_031 = channel_data3[0]
    channel_032 = channel_data3[1]
    channel_033 = channel_data3[2]

    cannal_031 = []
    cannal_032 = []
    cannal_033 = []

    c_0 = channel_031[8:]
    c_0 = np.asarray(c_0)
    for i in range(495):
        for j in range(4):
            cannal_031.append(c_0[i][j])

    c_0 = channel_032[8:]
    c_0 = np.asarray(c_0)
    for i in range(495):
        for j in range(4):
            cannal_032.append(c_0[i][j])

    c_0 = channel_033[8:]
    c_0 = np.asarray(c_0)
    for i in range(495):
        for j in range(4):
            cannal_033.append(c_0[i][j])

    caracte = calculo_caracte2(cannal_031)
    x = redderecha_izquierda(caracte)

    if x == 0:

        distancia = arduino2.readline()

        print("DERECHA")
        arduino1.write(b'6')

        if distancia == "Continue":
            arduino1.write(b'6')

        elif distancia == "Stop":
            arduino1.write(b'1')

        sleep(4)
        arduino1.write(b'1')

    elif x == 1:

        distancia = arduino2.readline()

        print("IZQUIERDA")
        arduino1.write(b'5')

        if distancia == "Continue":
            arduino1.write(b'5')

        elif distancia == "Stop":
            arduino1.write(b'1')

        sleep(4)
        arduino1.write(b'1')

    sleep(2)
    # LECTURA DE ARRIBA

    for line in book5_text:
        engine.say(line)
        engine.runAndWait()

    sleep(8)

    print("buscando la transmicion EEG de arriba")
    streams = resolve_stream('type','EEG')

    inlet = StreamInlet(streams[0])

    channel_data = {}

    for i in range(interacciones):
        for i in range(3):
            sample, timestamp = inlet.pull_sample()
            if i not in channel_data:
                channel_data[i] = sample 
            else:
                channel_data[i].append(sample[:f_max])

        fps_counter.append(time.time() - last_print)
        last_print = time.time()
        cur_raw_hz = 1/(sum(fps_counter)/len(fps_counter))

    channel_01 = channel_data[0]
    channel_02 = channel_data[1]
    channel_03 = channel_data[2]

    cannal_01 = []
    cannal_02 = []
    cannal_03 = []

    c_0 = channel_01[8:]
    c_0 = np.asarray(c_0)
    for i in range(495):
        for j in range(4):
            cannal_01.append(c_0[i][j])

    c_0 = channel_02[8:]
    c_0 = np.asarray(c_0)
    for i in range(495):
        for j in range(4):
            cannal_02.append(c_0[i][j])

    c_0 = channel_03[8:]
    c_0 = np.asarray(c_0)
    for i in range(495):
        for j in range(4):
            cannal_03.append(c_0[i][j])

    caracte = calculo_caracte2(cannal_021)
    x = redArriba_abajo(caracte)

    if x == 1:

        distancia = arduino2.readline()

        print("ARRIBA")
        arduino1.write(b'2')

        if distancia == "Continue":
            arduino1.write(b'2')

        elif distancia == "Stop":
            arduino1.write(b'1')

        sleep(4)
        arduino1.write(b'1')

    elif x == 0:

        distancia = arduino2.readline()

        print("ABAJO")
        arduino1.write(b'3')

        if distancia == "Continue":
            arduino1.write(b'3')

        elif distancia == "Stop":
            arduino1.write(b'1')


        sleep(4)
        arduino1.write(b'1')

    sleep(2)


buscando la transmicion EEG de la derecha
1/1 [==============================] - 1s 917ms/step
DERECHA
buscando la transmicion EEG de abajo
1/1 [==============================] - 0s 70ms/step
ARRIBA
buscando la transmicion EEG de la izquierda
1/1 [==============================] - 0s 67ms/step
IZQUIERDA
buscando la transmicion EEG de arriba
1/1 [==============================] - 0s 77ms/step
ARRIBA


KeyboardInterrupt: 